In [111]:
import numpy as np

# Calculate the probability that the model will be at a particular hidden state at time t
# Dynamic algorithm that contains for each time step, the probability that the all possible combination of states occured before with the observations times the probability of the specific state and its observed data
# o -> observations
def forward(o, p_transition, p_emission, pi):

    # Start by intitializing a vector of the alphas. This will contain the probabilities of all states at all time t
    # Create zero vectore that has rows the length of the observations and columns length as states in the model
    # alpha -> forward state probabilities
    alpha = np.zeros((o.shape[0], p_transition.shape[0]))

    # Set the initial probabilities
    alpha[0, :] = pi * p_emission[:, o[0]]

    # For all observations ...
    for t in range(1, o.shape[0]):
        # for all states s_t ...
        # (This could be further vectorized)
        for s_t in range(p_transition.shape[0]):
            # Find the probabilities of the transitions to the states based on previos alpha stored values
            # Then multiply by the probability of the observed emission occurring 
            # 
            # Vectorized - First part is the summation over all states
            alpha[t, s_t] = p_emission[s_t, o[t]] * alpha[t - 1] @ (p_transition[:, s_t])

    # So now you basically have whats the probability of seeing each state per time t GIVEN the observable data
    return alpha

# The same concept but the exact opposite of the forward algorithm
# So given a state, what is the probability you see the future data
def backward(o, p_transition, p_emission):

    # Start by intitializing a vector of the alphas. This will contain the probabilities of all states at all time t
    # Create zero vectore that has rows the length of the observations and columns length as states in the model
    # beta -> backward state probabilities
    beta = np.zeros((o.shape[0], p_transition.shape[0]))

    # With Beta, you have to start backward, as that is the only way it makes sense
    # So according to the formula, you have to set the last beta value as 1
    # Not sure why, but probably just as a starter *** ?
    beta[-1] = np.ones((p_transition.shape[0]))

    # Then start looping backward from -2 index
    # So for all observations ...
    for t in range(o.shape[0] - 2, -1, -1): # The 2nd index for stop is -1; excluding
        # for all states s_t
        # (This could be further vectorized)
        for s_t in range(p_transition.shape[0]):
            # Find the the "future" alphas will emit the observed data and multiply it by the probability that it will transition there
            # I 90% understand why its doing this. Look at this again ***
            beta[t, s_t] = (beta[t + 1] * p_emission[:, o[t + 1]]).dot(p_transition[s_t, :])

    # Returns probability of observing future data given the state at time t
    return beta


# We cannot see the "hidden" states, so this algorithm tries to emulate Maximum Likelihood Estimate
# In this view, the transition matrix for hidden state_1 to state_j would be (Expected # of transitions from state_i to state_j) / (Total transitions out of state_i)
# The estimated emission matrix given a state_i and observation_o is (Expected # of times state_i output observation_o) / (Total # of times state_i output an observation)
# Semi understand what is going on logically, look at this later ***
def baum_welch(o, p_transition, p_emission, pi, n=100):
    # ALl stuff to iterate over for each step in the algorighm
    # Number of states
    M = p_transition.shape[0]
    # Number of observable data
    T = o.__len__()

    for _ in range(n):
        # Get the forward and backward you need for the E-step
        alpha = forward(o, p_transition, p_emission, pi)
        beta = backward(o, p_transition, p_emission)
        
        # This is the E step. In the E steo, you calculate the Xi and the gamma for the M step for all time points
        xi = np.zeros((M, M, T - 1))
        # We first get all the Xi
        for t in range(T - 1):
            # The lower part of equation vectorized. Includes the probability of getting to the states, the transitioning to the next state then seeing the future observations for all possible combilations of states at each time step
            xi_denom = ( (alpha[t, :].T @ p_transition) * p_emission[:, o[t + 1]].T ) @ beta[t + 1, :]
            # For all states ...
            for i in range(M):
                # This is given all possible states that I am in, what is the probability to transition into the next state_j - vectorized
                xi_num = alpha[t, i] * p_transition[i, :] * p_emission[:, o[t + 1]].T * beta[t + 1, :].T
                # Once both above, divide
                xi[i, :, t] = xi_num / xi_denom

        # This is the gamma part of the E-step
        # quite simple to implement as you sum over all xi, but it is for all states in state_j and not state_i, so i summed over axis 1 not zero *** check
        gamma = np.sum(xi, axis=1)

        # This is the first part of the M-step
        p_transition = np.sum(xi, axis=2) / np.sum(gamma, axis=1).reshape((-1, 1)) # reshape to make it 2 dimentional be its just all the values in vector form and not list form

        # Keep getting this error: IndexError: boolean index did not match indexed array along dimension 1; dimension is 299 but corresponding boolean dimension is 300
        # So I assume I need to add another gamma. Idk why 
        # Oh, this is not needed for p_transition as that only goes up till T-1 but for p_emissions, it goes up to T
        # where to I get the last T values though?
        # Add additional T'th values in gamma because we went up toll T - 1 as same as above
        # T - 1 is 0s
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=1).reshape((-1, 1))))

        # Sum over all time
        denominator = np.sum(gamma, axis=1)
        denominator = denominator.reshape((-1, 1))
        # For all possible emissions
        for m in range(p_emission.shape[1]):
            # generate a mask to only index the emission values for when I am in the correct emission value
            bool_mask = (o == m)
            # Get each emission value for the right observed value
            p_emission[:, m] = np.sum(gamma[:, bool_mask], axis=1)

        p_emission = np.divide(p_emission, denominator)

    return p_transition, p_emission


In [112]:
import numpy as np
from scipy.stats import multinomial
# Generate Data

# Initial state probabilities
pi = np.array([0.5, 0.5])

# Transition Probabilities for the state
# 2 states
p_transition = np.array([
    [0.6, 0.4],
    [0.4, 0.6]
])

# Emission Probabilities
# 3 Observable variables
p_emission = np.array([
    [0.2, 0.4, 0.4],
    [0.5, 0.1, 0.4]
])

def generate_data_multinomial(states, p_transition, p_emission, n=100, pi=None, random_state=42):
    np.random.seed(random_state)
    
    # Generate an initial state probability if no pi given
    if type(pi) == "NoneType":
        pi = np.random.dirichlet(np.ones(states))
    
    # A list of the states
    s = []
    
    # First get the initial state
    s.append(np.argmax(multinomial.rvs(n=1, p=pi)))
    
    # Generate all the other states
    for _ in range(1, n):
        s.append(
            np.argmax(
                # The probability is based on the last state
                multinomial.rvs(n=1, p=p_transition[s[-1]])
                )
            )
    
    # A list of all the emissions
    e = []
    
    # Generate all the emissions
    for n in range(n):
        e.append(
            np.argmax(
                # This is the same as for the generating states but with p_emission
                multinomial.rvs(n=1, p=p_emission[s[-1]])
            )
        )
    
    # Return the states and emissions
    return s, e

In [122]:
# Initial state probabilities
pi = np.array([0.5, 0.5])

# Transition Probabilities for the state
# 2 states
p_transition = np.array([
    [0.6, 0.4],
    [0.4, 0.6]
])

# Emission Probabilities
# 3 Observable variables
p_emission = np.array([
    [0.2, 0.4, 0.4],
    [0.5, 0.1, 0.4]
])
s, e = generate_data_multinomial(2, p_transition, p_emission, n=300, pi=pi)

In [123]:
p_transition_matrix, p_emission_matrix = baum_welch(np.array(e), p_transition, p_emission, pi)

In [124]:
p_emission_matrix

array([[0.05229412, 0.41352448, 0.5341814 ],
       [0.40446124, 0.36412521, 0.23141356]])